In [1]:
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils import data
import tensorflow as tf
from tqdm import tqdm,trange
from transformers import *
# from ner_eval import classification_report,csv_report,f1_score
from seqeval.scheme import IOB2
from seqeval.metrics import classification_report,f1_score

from sklearn.model_selection import KFold

In [2]:
data_train = pd.read_csv("BC_train_IOB2.txt",sep = '\t')
data_dev = pd.read_csv("BC_dev_IOB2.txt",sep = '\t')

In [3]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                     s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence#").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [4]:
getter = SentenceGetter(data_train)
dev_getter = SentenceGetter(data_dev)

In [5]:
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
dev_sentences = [[word[0] for word in sentence] for sentence in dev_getter.sentences]

In [6]:
labels = [[s[1] for s in sent] for sent in getter.sentences]
dev_labels = [[s[1] for s in sent] for sent in dev_getter.sentences]

In [7]:
tag_values = list(set(data_train["Tag"].values))
tag_values.append("PAD")
tag_values

['B-', 'I-', 'O', 'PAD']

In [ ]:
bs = 16
model_ty = 'bert-base-multilingual-cased'
# tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizer.from_pretrained(model_ty, do_lower_case=False)